In [2]:
!pip install langchain-groq langchain-nvidia-ai-endpoints langchain langchain-community langgraph pydantic duckduckgo-search langchain-tavily

## Basic ChatBOT

In [9]:
from langgraph.graph import END, StateGraph, add_messages
# from langchain_groq import ChatGroq
# from langchain_tavily import TavilySearch
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_core.messages import AIMessage, HumanMessage
from typing import TypedDict, Annotated

In [10]:
from kaggle_secrets import UserSecretsClient
nvidia_model_name = UserSecretsClient().get_secret('NVIDIA_MODEL_NAME')
nvidia_api_key = UserSecretsClient().get_secret('NVIDIA_API_KEY')
tavily_api_key = UserSecretsClient().get_secret("TAVILY_API_KEY")

In [5]:
search_tool = DuckDuckGoSearchRun()

In [6]:
search_tool.invoke("When was the Spacex's latest launch?")

/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:


"2 days ago · Live updates from Sunday's SpaceX KF-02 mission that scrubbed with 30 seconds left in the countdown at Cape Canaveral Space Force Station in Florida. 1 day ago · SpaceX launched a batch of Amazon's Project Kuiper broadband satellites on Monday (Aug. 11). It was the 100th liftoff of the year already for Elon Musk's company. 1 day ago · The Vandenberg Space Force Base announced that a SpaceX Falcon 9 launch scheduled for the evening of Monday, Aug. 11 has been rescheduled to Tuesday, Aug. 12. SpaceX, which has updated its launch ... 1 day ago · SpaceX was not one of Amazon's original choices to fly its Project Kuiper broadband internet satellites, but Elon Musk's company just knocked out its second launch in less than a month for the ... Mar 6, 2025 · Seven weeks after an explosion sent pieces of a Starship launch vehicle falling into the Atlantic, SpaceX on Thursday again launched its largest and most powerful rocket before losing contact with Starship several minutes into

In [7]:
llm = ChatNVIDIA(
    model = nvidia_model_name,
    api_key = nvidia_api_key,
)

In [11]:
class BasicChatState(TypedDict):
    messages: Annotated[list, add_messages]

In [12]:
def chatbot(state: BasicChatState):
    return {
        "messages": [llm.invoke(state["messages"])]
    }

In [14]:
graph = StateGraph(BasicChatState)

In [15]:
graph.add_node("chatbot", chatbot)

In [16]:
graph.set_entry_point("chatbot")

In [17]:
graph.add_edge("chatbot", END)

In [18]:
app = graph.compile()

In [23]:
while True:
    user_input = input("User: ")
    if (user_input in ["exit", "end"]):
        break
    else:
        result = app.invoke({
            "messages": [HumanMessage(content = user_input)]
        })
        print(result)

User:  exit


## Building Bot with tools

In [24]:
tools= [search_tool]

In [25]:
llm_with_tools = llm.bind_tools(tools = tools)

In [28]:
class BasicChatBot(TypedDict):
    messages: Annotated[list, add_messages]

In [29]:
def chatbot(state: BasicChatBot):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

In [49]:
def tools_router(state: BasicChatBot):
    last_message = state["messages"][-1]

    if ( hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0 ):
        return "tools"
    else:
        return END

In [50]:
from langgraph.prebuilt import ToolNode 

In [51]:
tool_node = ToolNode(tools = tools)

In [52]:
graph = StateGraph(BasicChatBot)

In [53]:
graph.add_node("chatbot", chatbot)

In [54]:
graph.set_entry_point("chatbot")

In [55]:
graph.add_node("tools", tool_node)

In [56]:
graph.add_conditional_edges("chatbot", tools_router)

In [57]:
graph.add_edge("tools", "chatbot")

In [58]:
app = graph.compile()

In [59]:
while True:
    user_input = input("User: ")
    if (user_input in ["exit", "end"]):
        break
    else:
        result = app.invoke({
            "messages": [HumanMessage(content = user_input)]
        })
        print(result)

User:  when the SpaceX's latest launch?


/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e34e84aab30>


{'messages': [HumanMessage(content="when the SpaceX's latest launch?", additional_kwargs={}, response_metadata={}, id='74fcc1f1-f1e1-46b2-b9f6-db55d9eef8a7'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-50a3451b4c0b45f2bf0cb5e2d1bf96ee', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query": "SpaceX latest launch"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-50a3451b4c0b45f2bf0cb5e2d1bf96ee', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query": "SpaceX latest launch"}'}}], 'token_usage': {'prompt_tokens': 302, 'total_tokens': 318, 'completion_tokens': 16}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-70b-instruct'}, id='run--a52e4e68-8616-4001-9972-8b2dc9b36155-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'SpaceX latest launch'}, 'id': 'chatcmpl-tool-50a3451b4c0b45f2bf0cb5e2d1bf96ee', 'type': 'tool_c

User:  Whatv is the current weather in bangalore?


/usr/local/lib/python3.11/dist-packages/langchain_community/utilities/duckduckgo_search.py:63: RuntimeWarning: This package (`duckduckgo_search`) has been renamed to `ddgs`! Use `pip install ddgs` instead.
  with DDGS() as ddgs:
sys:1: ResourceWarning: Unclosed socket <zmq.Socket(zmq.PUSH) at 0x7e34e852a4a0>


{'messages': [HumanMessage(content='Whatv is the current weather in bangalore?', additional_kwargs={}, response_metadata={}, id='35526c67-9712-47b3-9446-4091d640626a'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'chatcmpl-tool-d8403b05aeb64e4ea66492f23a57feb5', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query": "current weather in Bangalore"}'}}]}, response_metadata={'role': 'assistant', 'content': None, 'tool_calls': [{'id': 'chatcmpl-tool-d8403b05aeb64e4ea66492f23a57feb5', 'type': 'function', 'function': {'name': 'duckduckgo_search', 'arguments': '{"query": "current weather in Bangalore"}'}}], 'token_usage': {'prompt_tokens': 305, 'total_tokens': 321, 'completion_tokens': 16}, 'finish_reason': 'tool_calls', 'model_name': 'meta/llama-3.1-70b-instruct'}, id='run--d6b1c977-54be-48a3-9d4f-7a188000edf3-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current weather in Bangalore'}, 'id': 'chatcmpl-tool-d8403b05aeb64e4e

User:  exit


## Building memory